<a href="https://colab.research.google.com/github/MthabisiPatrice/Machine-Learning-/blob/main/taxi_hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset Description

This dataset is designed to predict taxi trip fares based on various factors such as distance, time of day, traffic conditions, and more. It provides realistic synthetic data for regression tasks, offering a unique opportunity to explore pricing trends in the taxi industry.

Key Features

Distance (in kilometers): The length of the trip.
Pickup Time: The starting time of the trip.
Dropoff Time: The ending time of the trip.
Traffic Condition: Categorical indicator of traffic (light, medium, heavy).
Passenger Count: Number of passengers for the trip.
Weather Condition: Categorical data for weather (clear, rain, snow).
Trip Duration (in minutes): Total trip time.
Fare Amount (target): The cost of the trip (in USD).

In [3]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn import tree
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, r2_score
import altair as alt
from geopy.distance import geodesic
import math

In [4]:
taxi = pd.read_csv('/content/taxi_data.csv')

In [5]:
taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Trip_Distance_km       950 non-null    float64
 1   Time_of_Day            950 non-null    object 
 2   Day_of_Week            950 non-null    object 
 3   Passenger_Count        950 non-null    float64
 4   Traffic_Conditions     950 non-null    object 
 5   Weather                950 non-null    object 
 6   Base_Fare              950 non-null    float64
 7   Per_Km_Rate            950 non-null    float64
 8   Per_Minute_Rate        950 non-null    float64
 9   Trip_Duration_Minutes  950 non-null    float64
 10  Trip_Price             951 non-null    float64
dtypes: float64(7), object(4)
memory usage: 86.1+ KB


In [6]:
import pandas as pd

# Fill missing values for numerical columns with their mean
taxi['Trip_Distance_km'] = taxi['Trip_Distance_km'].fillna(taxi['Trip_Distance_km'].mean())
taxi['Passenger_Count'] = taxi['Passenger_Count'].fillna(taxi['Passenger_Count'].mean())
taxi['Base_Fare'] = taxi['Base_Fare'].fillna(taxi['Base_Fare'].mean())
taxi['Per_Km_Rate'] = taxi['Per_Km_Rate'].fillna(taxi['Per_Km_Rate'].mean())
taxi['Per_Minute_Rate'] = taxi['Per_Minute_Rate'].fillna(taxi['Per_Minute_Rate'].mean())
taxi['Trip_Duration_Minutes'] = taxi['Trip_Duration_Minutes'].fillna(taxi['Trip_Duration_Minutes'].mean())
taxi['Trip_Price'] = taxi['Trip_Price'].fillna(taxi['Trip_Price'].mean())

# Fill missing values for categorical columns with their mode (most frequent value)
taxi['Time_of_Day'] = taxi['Time_of_Day'].fillna(taxi['Time_of_Day'].mode()[0])
taxi['Day_of_Week'] = taxi['Day_of_Week'].fillna(taxi['Day_of_Week'].mode()[0])
taxi['Traffic_Conditions'] = taxi['Traffic_Conditions'].fillna(taxi['Traffic_Conditions'].mode()[0])
taxi['Weather'] = taxi['Weather'].fillna(taxi['Weather'].mode()[0])

# Check the result
print(taxi.isnull().sum())



Trip_Distance_km         0
Time_of_Day              0
Day_of_Week              0
Passenger_Count          0
Traffic_Conditions       0
Weather                  0
Base_Fare                0
Per_Km_Rate              0
Per_Minute_Rate          0
Trip_Duration_Minutes    0
Trip_Price               0
dtype: int64


In [7]:
taxi.head()

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.350000,Morning,Weekday,3.0,Low,Clear,3.560000,0.80,0.32,53.82,36.262400
1,47.590000,Afternoon,Weekday,1.0,High,Clear,3.502989,0.62,0.43,40.57,56.874773
2,36.870000,Evening,Weekend,1.0,High,Clear,2.700000,1.21,0.15,37.27,52.903200
3,30.330000,Evening,Weekday,4.0,Low,Clear,3.480000,0.51,0.15,116.81,36.469800
4,27.070547,Evening,Weekday,3.0,High,Clear,2.930000,0.63,0.32,22.64,15.618000


In [8]:
# first we see the kind of data before
for column in ['Traffic_Conditions', 'Time_of_Day', 'Day_of_Week', 'Weather']:
    unique_values = taxi[column].unique()
    print(f"Unique values in '{column}': {unique_values}")

Unique values in 'Traffic_Conditions': ['Low' 'High' 'Medium']
Unique values in 'Time_of_Day': ['Morning' 'Afternoon' 'Evening' 'Night']
Unique values in 'Day_of_Week': ['Weekday' 'Weekend']
Unique values in 'Weather': ['Clear' 'Rain' 'Snow']


In [9]:
#now we onehot encode the nonsensical data.
one_hot_encoded_data = pd.get_dummies(taxi, columns=['Traffic_Conditions', 'Time_of_Day', 'Day_of_Week', 'Weather'])

In [10]:
taxi_encoded = one_hot_encoded_data

In [11]:
#creating a
x = taxi_encoded.drop('Trip_Price', axis=1)
y = taxi_encoded['Trip_Price']

In [12]:
#train the model.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
# convert data to d matrix
dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)

In [14]:
#define the parameters

xg_reg = xgb.XGBRegressor(
    objective='reg:squarederror',
    colsample_bytree=0.5,
    learning_rate=0.1,
    max_depth=4,
    alpha=2,
    n_estimators=500,
    subsample=0.8,
    min_child_weight=5,
    gamma=2
)

In [15]:
#train the model

model = xg_reg.fit(x_train, y_train)

In [17]:
#predictions
predictions = model.predict(x_test)

In [18]:
#model perfomance
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 70.139769384495
R-squared: 0.9155313019204472
